In [87]:
import numpy as np
import csv
import pandas as pd
from sklearn.linear_model import LinearRegression
from numpy.linalg import norm
from sklearn.linear_model import Ridge

In [59]:
def distance_calculator(main,other):
    return np.sum((np.subtract(main,other)**2).values.flatten())
    

In [60]:
def find_closest_neighbour(agentpos,others):
    min_dist=np.inf
    jar=agentpos
    agent=others[names[2*int(agentpos):2*int(agentpos)+2]]
    for i in range(10):
        if i!=int(agentpos):
            d=distance_calculator(agent,others[names[2*i:2*i+2]])
            if d<min_dist:
                min_dist=d
                jar=i
    return jar

In [61]:
def newprocessX(trainsize=100):
    newdf=np.empty((0,44))
    for i in range(trainsize):
        cs1=pd.read_csv("train/X/X_"+str(i)+".csv")
        cs2=cs1.eq(' agent',axis=1).any(axis=0)
        agentpos=cs2[cs2].index[0][-1]
        cs2=cs1[[' x'+agentpos,' y'+agentpos]]
        jar=find_closest_neighbour(agentpos,cs1[names])
        nearest_neigh=cs1[names[2*jar:2*jar+2]]
        cs2=np.array(cs2.values.flatten())
        cs2=np.append(cs2,nearest_neigh.values.flatten())
        newdf=np.append(newdf,[cs2],axis=0)
        #newdf=np.append(newdf,[nearest_neigh.values.flatten()])
        
    return newdf



In [62]:
names=[]
for i in range(10):
    names.append(' x'+str(i))
    names.append(' y'+str(i))

In [63]:
def processX(trainsize=100):
    newdf=np.empty((0,22))
    for i in range(trainsize):
        cs1=pd.read_csv("train/X/X_"+str(i)+".csv")
        cs2=cs1.eq(' agent',axis=1).any(axis=0)
        agentpos=cs2[cs2].index[0][-1]
        cs2=cs1[[' x'+agentpos,' y'+agentpos]]
        cs2=np.array(cs2.values.flatten())
        newdf=np.append(newdf,[cs2],axis=0)
        
    return newdf


In [64]:
def processy(trainsize=100):
    newdf=np.empty((0,60))
    for i in range(trainsize):
        cs1=pd.read_csv("train/y/y_"+str(i)+".csv",usecols=[' x',' y'])
        cs2=np.array(cs1.values.flatten())
        cs2len=len(cs2)
        cs2last=cs2[-1]
        for j in range(60-cs2len):
            cs2=np.append(cs2,cs2last)
        newdf=np.append(newdf,[cs2],axis=0)
        
    return newdf



In [65]:
def valXprocess(testsize=100,flag=0):
    if flag==0:
        newdf=np.empty((0,22))
        for i in range(testsize):
            cs1=pd.read_csv("val/X/X_"+str(i)+".csv")
            cs2=cs1.eq(' agent',axis=1).any(axis=0)
            agentpos=cs2[cs2].index[0][-1]
            cs2=cs1[[' x'+agentpos,' y'+agentpos]]
            cs2=np.array(cs2.values.flatten())
            newdf=np.append(newdf,[cs2],axis=0)
    else:
        newdf=np.empty((0,22))
        for i in range(testsize):
            cs1=pd.read_csv("test/X/X_"+str(i)+".csv")
            cs2=cs1.eq(' agent',axis=1).any(axis=0)
            agentpos=cs2[cs2].index[0][-1]
            cs2=cs1[[' x'+agentpos,' y'+agentpos]]
            cs2=np.array(cs2.values.flatten())
            newdf=np.append(newdf,[cs2],axis=0)
        
        
    return newdf



In [66]:
def newvalXprocess(testsize=100,flag=0):
    if flag==0:
        newdf=np.empty((0,44))
        for i in range(testsize):
            cs1=pd.read_csv("val/X/X_"+str(i)+".csv")
            cs2=cs1.eq(' agent',axis=1).any(axis=0)
            agentpos=cs2[cs2].index[0][-1]
            cs2=cs1[[' x'+agentpos,' y'+agentpos]]
            jar=find_closest_neighbour(agentpos,cs1[names])
            nearest_neigh=cs1[names[2*jar:2*jar+2]]
            cs2=np.array(cs2.values.flatten())
            cs2=np.append(cs2,nearest_neigh.values.flatten())
            newdf=np.append(newdf,[cs2],axis=0)
            #newdf=np.append(newdf,[nearest_neigh.values.flatten()])
    else:
        newdf=np.empty((0,44))
        for i in range(testsize):
            cs1=pd.read_csv("test/X/X_"+str(i)+".csv")
            cs2=cs1.eq(' agent',axis=1).any(axis=0)
            agentpos=cs2[cs2].index[0][-1]
            cs2=cs1[[' x'+agentpos,' y'+agentpos]]
            jar=find_closest_neighbour(agentpos,cs1[names])
            nearest_neigh=cs1[names[2*jar:2*jar+2]]
            cs2=np.array(cs2.values.flatten())
            cs2=np.append(cs2,nearest_neigh.values.flatten())
            newdf=np.append(newdf,[cs2],axis=0)

    return newdf



In [67]:
def valyprocess(testsize=100):
    newdf=np.empty((0,60))
    for i in range(testsize):
        cs1=pd.read_csv("val/y/y_"+str(i)+".csv",usecols=[' x',' y'])
        cs2=np.array(cs1.values.flatten())
        cs2len=len(cs2)
        cs2last=cs2[-1]
        for j in range(60-cs2len):
            cs2=np.append(cs2,cs2last)
        newdf=np.append(newdf,[cs2],axis=0)
        
    return newdf



In [71]:
def flatten_weights(weights):
    return np.concatenate([w.flatten() for w in sum(weights,())])

def unflatten_weights(weights_flat, layer_sizes):
    weights = list()
    counter = 0
    for i in range(len(layer_sizes)-1):
        W_size = layer_sizes[i+1] * layer_sizes[i]
        b_size = layer_sizes[i+1]
        
        W = np.reshape(weights_flat[counter:counter+W_size], (layer_sizes[i+1], layer_sizes[i]))
        counter += W_size

        b = weights_flat[counter:counter+b_size][None]
        counter += b_size

        weights.append((W,b))  
    return weights

def log_sum_exp(Z):
    Z_max = np.max(Z,axis=1)
    return Z_max + np.log(np.sum(np.exp(Z - Z_max[:,None]), axis=1)) # per-colmumn max

class NeuralNet():
    # uses sigmoid nonlinearity
    def __init__(self, hidden_layer_sizes, lammy=1, max_iter=1000):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.lammy = lammy
        self.max_iter = max_iter

    def funObj(self, weights_flat, X, y):
        n,d=X.shape
        #randindx=np.random.randint(0,n,size=500) #choose 500 random indices
        #X=X[randindx,:] #select those subsets of X
        #y=y[randindx]  #select those subsets of y
        weights = unflatten_weights(weights_flat, self.layer_sizes)
        
        activations = [X]
        for W, b in weights:
            Z = X @ W.T + b
            X = 1/(1+np.exp(-Z))
            activations.append(X)

        yhat = Z
        
       
        if self.classification: # softmax- TODO: use logsumexp trick to avoid overflow
            tmp = np.sum(np.exp(yhat), axis=1)
            # f = -np.sum(yhat[y.astype(bool)] - np.log(tmp))
            f = -np.sum(yhat[y.astype(bool)] - log_sum_exp(yhat))
            grad = np.exp(yhat) / tmp[:,None] - y
        else:  # L2 loss
            f = 0.5*np.sum((yhat-y)**2)  
            grad = yhat-y # gradient for L2 loss

        grad_W = grad.T @ activations[-2]  
        grad_b = np.sum(grad, axis=0)  

        g = [(grad_W, grad_b)]

        for i in range(len(self.layer_sizes)-2,0,-1):
            W, b = weights[i]
            grad = grad @ W
            grad = grad * (activations[i] * (1-activations[i])) # gradient of logistic loss
            grad_W = grad.T @ activations[i-1]
            grad_b = np.sum(grad,axis=0)  

            g = [(grad_W, grad_b)] + g # insert to start of list

        g = flatten_weights(g)
        
        # add L2 regularization
        f += 0.5 * self.lammy * np.sum(weights_flat**2)
        g += self.lammy * weights_flat 
        
        return f, g

    
    def fit(self, X, y):
        if y.ndim == 1:
            y = y[:,None]
            
        self.layer_sizes = [X.shape[1]] + self.hidden_layer_sizes + [y.shape[1]]
        self.classification = y.shape[1]>1 # assume it's classification iff y has more than 1 column

        # random init
        scale = 0.01
        weights = list()
        for i in range(len(self.layer_sizes)-1):
            W = scale * np.random.randn(self.layer_sizes[i+1],self.layer_sizes[i])
            b = scale * np.random.randn(1,self.layer_sizes[i+1])
            weights.append((W,b))
        weights_flat = flatten_weights(weights)
        
        # utils.check_gradient(self, X, y, len(weights_flat), epsilon=1e-6)
        epoch=100
        n,d=X.shape
        self.weights=unflatten_weights(weights_flat,self.layer_sizes)
        for i in range(epoch):
            weights_flat=flatten_weights(self.weights)
            weights_flat_new, f = RegularfindMin(self.funObj, weights_flat, self.max_iter, X, y, verbose=True)
           
            self.weights = unflatten_weights(weights_flat_new, self.layer_sizes)


    def predict(self, X):
        for W, b in self.weights:
            Z = X @ W.T + b
            X = 1/(1+np.exp(-Z))
        if self.classification:
            return np.argmax(Z,axis=1)
        else:
            return Z






In [72]:
def RegularfindMin(funObj, w, maxEvals, *args, verbose=0):
    """
    Uses gradient descent to optimize the objective function

    This uses quadratic interpolation in its line search to
    determine the step size alpha
    """
    # Parameters of the Optimization
    optTol = 1e-2
    gamma = 1e-4

    # Evaluate the initial function value and gradient
    f, g = funObj(w,*args)
    funEvals = 1

    alpha = 1.
    while True:
        # Line-search using quadratic interpolation to find an acceptable value of alpha
        gg = g.T@g

        while True:
            w_new = w - alpha * g
            f_new, g_new = funObj(w_new, *args)
            #print("find min ", w_new.shape)
            funEvals += 1

            if f_new <= f - gamma * alpha*gg:
                break

            if verbose > 1:
                print("f_new: %.3f - f: %.3f - Backtracking..." % (f_new, f))

            # Update step size alpha
            alpha = (alpha**2) * gg/(2.*(f_new - f + alpha*gg))

        # Print progress
       # if verbose > 0:
       #     print("%d - loss: %.3f" % (funEvals, f_new))

        # Update step-size for next iteration
        y = g_new - g
        alpha = -alpha*(y.T@g) / (y.T@y)

        # Safety guards
        if np.isnan(alpha) or alpha < 1e-10 or alpha > 1e10:
            alpha = 1.

        if verbose > 1:
            print("alpha: %.3f" % (alpha))

        # Update parameters/function/gradient
        w = w_new
        f = f_new
        g = g_new

        # Test termination conditions
        optCond = norm(g, float('inf'))

        if optCond < optTol:
            #if verbose:
             #   print("Problem solved up to optimality tolerance %.3f" % optTol)
            break

        if funEvals >= maxEvals:
           # if verbose:
            #    print("Reached maximum number of function evaluations %d" % maxEvals)
            break

    return w, f


In [73]:
def findMin(funObj, w, maxEvals, *args, verbose=0):
   
    # Parameters of the Optimization
    optTol = 1e-2
    gamma = 1e-4

    # Evaluate the initial function value and gradient
    f, g = funObj(w,*args)
    funEvals = 1
    count=0
    alpha = 0.001
    while True:
        gg = g.T.dot(g)
        w_new = w - alpha * g
        f_new, g_new = funObj(w_new, *args)
        funEvals += 1
        if np.abs(f_new-f) <= gamma * alpha*gg:
                break

        # Update parameters/function/gradient
        w = w_new
        f = f_new
        g = g_new

        # Test termination conditions
        optCond = norm(g, float('inf'))

        if optCond < optTol:
            #if verbose:
                #print("Problem solved up to optimality tolerance %.3f" % optTol)
            break

        if funEvals >= maxEvals:
            #if verbose:
            #    print("Reached maximum number of function evaluations %d" % maxEvals)
            break

    return w, f


In [95]:
def threesecpred(trainsize=1007,valshape=19,flag=0):
   # valshape += 1
    pred=np.zeros((valshape,60))
    for i in range(30):
        X=processX(trainsize)
        y=processy(trainsize)
        hidden_layer_sizes = [10]
        #regx = NeuralNet(hidden_layer_sizes)
        #regx=LinearRegression()
        regx = logRegL1()
        regx.fit(X, y[:,2*i])
        valx=valXprocess(testsize=valshape,flag=flag)
        #valy=valyprocess(testsize=valshape)
        pred[:,2*i]=regx.predict(valx).flatten()
        
        #print(pred)
        #print("score"+str(2*i),regx.score(valx,valy[:,2*i]))
        #regy=LinearRegression()
        #regy = NeuralNet(hidden_layer_sizes)
        regy = logRegL1()
        regy.fit(X, y[:,2*i+1])
        pred[:,2*i+1]=regy.predict(valx).flatten()
        #print("score"+str(2*i+1),regy.score(valx,valy[:,2*i+1]))
        #print(pred)
    return pred

In [ ]:
newpred=threesecpred(trainsize=2308,valshape=524,flag=0);

/Users/aks/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp


7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
218
224
267
314
363
414
461
510
559
608
657
706
755
804
853
902
949
997
3
4
5
6
8
13
15
16
17
20
21
24
25
28
29
32
33
36
37
40
41
44
45
48
49
52
53
56
57
60
61
64
65
68
69
72
73
76
77
80
81
84
85
88
89
90
91
93
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79

/Users/aks/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars


960
8
9
10
11
12
14
20
21
22
23
24
30
31
37
38
43
44
45
46
47
51
53
54
55
56
60
62
63
64
65
69
71
72
73
74
77
80
81
82
83
87
88
89
90
92
93
94
95
99
100
101
102
105
107
108
109
110
114
115
116
117
120
126
127
128
129
130
135
136
137
138
139
143
144
145
146
149
151
152
153
154
158
159
160
161
168
169
173
175
176
177
178
183
184
185
186
187
192
193
194
195
196
201
202
208
209
213
215
216
217
218
223
224
225
226
227
231
232
233
234
237
239
240
241
244
246
247
248
250
253
254
255
256
261
262
263
264
265
270
271
276
277
282
283
288
289
294
295
300
301
306
307
312
313
314
315
316
321
322
327
328
333
334
339
340
345
346
347
348
349
354
355
360
361
366
367
372
373
378
379
384
385
390
391
396
397
402
403
408
409
414
415
420
421
426
427
432
433
438
439
444
445
450
451
456
457
462
463
468
469
474
475
480
481
486
487
492
493
498
499
504
505
510
511
516
517
522
523
528
529
534
535
540
541
546
547
552
553
558
559
564
565
569
570
571
572
573
576
578
579
580
581
586
587
591
592
593
598
599
605
606
610

/Users/aks/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


9
10
11
12
13
14
18
19
20
21
25
27
28
29
30
38
39
40
41
42
43
44
45
49
51
52
53
54
56
64
65
66
67
69
70
71
72
76
78
79
80
81
82
90
91
92
95
98
99
100
101
109
110
114
116
117
118
119
124
125
126
127
129
137
138
142
145
146
147
148
153
154
155
156
157
158
159
162
169
170
171
172
173
179
180
181
182
183
188
189
190
191
194
197
198
199
200
202
209
210
211
212
213
219
220
221
222
223
228
229
230
231
232
234
235
236
240
242
243
244
246
250
251
252
253
254
255
259
266
267
268
269
270
276
277
278
279
280
285
286
287
288
289
291
292
293
294
300
301
307
308
309
310
311
317
318
323
324
325
326
327
329
330
331
332
338
339
345
346
347
348
349
355
356
361
362
363
364
365
367
374
375
381
382
388
389
390
391
392
398
399
404
405
406
407
408
410
417
418
423
424
425
426
427
432
433
434
435
436
442
443
449
450
456
457
462
463
464
465
466
471
472
473
474
475
481
482
488
489
494
495
496
497
498
503
504
505
506
507
512
513
514
515
516
522
523
529
530
535
536
537
538
539
545
546
551
552
553
554
555
561
562
56

In [93]:
newpred3=threesecpred(trainsize=2308,valshape=20,flag=1);

/Users/aks/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/Users/aks/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/Users/aks/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/Users/aks/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/Users/aks/opt/anaconda3/lib/python3.7/site-packages/pandas/core

/Users/aks/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/Users/aks/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/Users/aks/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/Users/aks/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/Users/aks/opt/anaconda3/lib/python3.7/site-packages/pandas/core

In [94]:
newpred4=newpred3.flatten()
np.savetxt("submission_ridge_regression.csv",newpred4,delimiter=",")

In [91]:
valy=valyprocess(testsize=524)
rsme=np.mean((valy-newpred)**2)

In [92]:
rsme

0.6129807265528479

In [ ]:
newpred1=newpred.flatten()

In [ ]:
valx = np.hstack((val, np.atleast_2d(predx).T))

In [ ]:
valy= np.hstack((valx, np.atleast_2d(predy).T))

In [ ]:
valnew = np.delete(valy, [0,1], 1)

In [ ]:
newpred.shape

In [ ]:
regx.predict(X)

In [ ]:
rsme

In [ ]:
newpred

In [ ]:
np.savetxt("submission_1.csv",newpred,delimiter=",")

In [ ]:
newpred.shape

In [ ]:
newpred1

In [ ]:
newdf=np.empty((0,22))
trainsize=1
for i in range(trainsize):
        cs1=pd.read_csv("train/X/X_"+str(i)+".csv")
        cs2=cs1.eq(' agent',axis=1).any(axis=0)
        agentpos=cs2[cs2].index[0][-1]
        cs2=cs1[[' x'+agentpos,' y'+agentpos]]
        cs2=np.array(cs2.values.flatten())
        newdf=np.append(newdf,[cs2],axis=0)

In [ ]:
newdf

In [ ]:
cs1[names]

In [ ]:
names

In [ ]:
 cs2=cs1.eq(' agent',axis=1).any(axis=0)

In [ ]:
 cs2=cs1[[' x'+agentpos,' y'+agentpos]]

In [ ]:
euclidean_dist_squared(cs2,cs1[names[2:4]])

In [ ]:
agentpos

In [ ]:
a=cs1[names[2:4]]

In [ ]:
b=cs2

In [ ]:
nearest_neigh=cs1[names[2*jar:2*jar+2]]
nearest_neigh.values.flatten()

In [ ]:
nearest_neigh.values.flatten()

In [ ]:
b

In [ ]:
names

In [ ]:
for i in range(10):
    print(names[2*i:2*i+2],"\n")

In [ ]:
a=newprocessX(trainsize=2)

In [ ]:
b=processX(trainsize=2)

In [ ]:
a[0,:]

In [ ]:
b[0,:]

In [ ]:
a[1,:]

In [ ]:
b[1,:]

In [ ]:
a.shape

In [ ]:
b.shape

In [84]:
def newthreesecpred(trainsize=1007,valshape=19,flag=0):
   # valshape += 1
    pred=np.zeros((valshape,60))
    for i in range(30):
        X=newprocessX(trainsize)
        y=processy(trainsize)
        hidden_layer_sizes = [100]
        regx = NeuralNet(hidden_layer_sizes)
        #regx=NeuralNet()
        regx.fit(X, y[:,2*i])
        valx=newvalXprocess(testsize=valshape,flag=flag)
        #valy=valyprocess(testsize=valshape)
        pred[:,2*i]=regx.predict(valx).flatten()
        
        #print(pred)
        #print("score"+str(2*i),regx.score(valx,valy[:,2*i]))
        #regy=LinearRegression()
        regy = NeuralNet(hidden_layer_sizes)
        regy.fit(X, y[:,2*i+1])
        pred[:,2*i+1]=regy.predict(valx).flatten()
        #print("score"+str(2*i+1),regy.score(valx,valy[:,2*i+1]))
        #print(pred)
    return pred

In [75]:
newpred2=newthreesecpred(trainsize=2307,valshape=524)
valy2=valyprocess(testsize=524)
rsme2=np.mean((valy2-newpred2)**2)

/Users/aks/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: overflow encountered in exp


In [76]:
rsme2

0.4901247753201451

In [85]:
newpred3=newthreesecpred(trainsize=2307,valshape=20,flag=1)
#valy2=valyprocess(testsize=524)
#rsme2=np.mean((valy2-newpred2)**2)

/Users/aks/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: overflow encountered in exp


In [86]:
newpred4=newpred3.flatten()
np.savetxt("submission_nn_neighbour_100hl.csv",newpred4,delimiter=",")

In [100]:
class logRegL1:
    # Logistic Regression
    def __init__(self,L1_constant=1, verbose=0, maxEvals=1000):
        self.verbose = verbose
        self.maxEvals = maxEvals
        self.bias = True
        self.L1_constant = L1_constant

    def funObj(self, w, X, y):
        
        yXw = y * X.dot(w)

        # Calculate the function value
        f = np.sum(np.log(1. + np.exp(-yXw))) 

        # Calculate the gradient value
        res = - y / (1. + np.exp(yXw))
        g = X.T.dot(res)

        return f, g

    def fit(self,X, y):
        n, d = X.shape

        # Initial guess
        self.w = np.zeros(d)
        #utils.check_gradient(self, X, y)
        (self.w, f) = findMinL1(self.funObj, self.w,self.L1_constant,
                                      self.maxEvals, X, y, verbose=self.verbose)
    def predict(self, X):
        return np.sign(X@self.w)


In [101]:
def findMinL1(funObj, w, L1_lambda, maxEvals, *args, verbose=0):
    """
    Uses the L1 proximal gradient descent to optimize the objective function

    The line search algorithm divides the step size by 2 until
    it find the step size that results in a decrease of the L1 regularized
    objective function
    """
    # Parameters of the Optimization
    optTol = 1e-2
    gamma = 1e-4

    # Evaluate the initial function value and gradient
    f, g = funObj(w,*args)
    funEvals = 1

    alpha = 1.
    proxL1 = lambda w, alpha: np.sign(w) * np.maximum(abs(w)- L1_lambda*alpha,0)
    L1Term = lambda w: L1_lambda * np.sum(np.abs(w))

    while True:
        gtd = None
        # Start line search to determine alpha
        while True:
            w_new = w - alpha * g
            w_new = proxL1(w_new, alpha)

            if gtd is None:
                gtd = g.T.dot(w_new - w)

            f_new, g_new = funObj(w_new, *args)
            funEvals += 1

            if f_new + L1Term(w_new) <= f + L1Term(w) + gamma*alpha*gtd:
                # Wolfe condition satisfied, end the line search
                break

            if verbose > 1:
                print("Backtracking... f_new: %.3f, f: %.3f" % (f_new, f))

            # Update alpha
            alpha /= 2.

        # Print progress
        if verbose > 0:
            print("%d - alpha: %.3f - loss: %.3f" % (funEvals, alpha, f_new))

        # Update step-size for next iteration
        y = g_new - g
        alpha = -alpha*np.dot(y.T,g) / np.dot(y.T,y)

        # Safety guards
        if np.isnan(alpha) or alpha < 1e-10 or alpha > 1e10:
            alpha = 1.

        # Update parameters/function/gradient
        w = w_new
        f = f_new
        g = g_new

        # Test termination conditions
        optCond = norm(w - proxL1(w - g, 1.0), float('inf'))

        if optCond < optTol:
            if verbose:
                print("Problem solved up to optimality tolerance %.3f" % optTol)
            break

        if funEvals >= maxEvals:
            if verbose:
                print("Reached maximum number of function evaluations %d" % maxEvals)
            break
        print(funEvals)
    return w, f
